In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
!pip install opencv-python

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
import tensorflow as tf
from tensorflow import keras
import cv2
import glob
from xml.etree import ElementTree

In [4]:
print('Tensorflow Version: {}'.format(tf.__version__))
print('Keras Version: {}'.format(keras.__version__))

In [5]:
annotations_directory = '../input/face-mask-detection/annotations'
images_directory = '../input/face-mask-detection/images'

In [6]:
annotations_files = !ls '../input/face-mask-detection/annotations'
annotations_files[:10]

In [7]:
images_files = !ls '../input/face-mask-detection/images'
images_files[:10]

In [8]:
len(annotations_files), len(images_files)

In [9]:
information = {'xmin': [], 'ymin': [], 'xmax': [], 'ymax': [], 'label': [], 'file': [], 'width': [], 'height': []}

for annotation in glob.glob(annotations_directory + '/*.xml'):
    tree = ElementTree.parse(annotation)
    
    for element in tree.iter():
        if 'size' in element.tag:
            for attribute in list(element):
                if 'width' in attribute.tag: 
                    width = int(round(float(attribute.text)))
                if 'height' in attribute.tag:
                    height = int(round(float(attribute.text)))    

        if 'object' in element.tag:
            for attribute in list(element):
                
                if 'name' in attribute.tag:
                    name = attribute.text                 
                    information['label'] += [name]
                    information['width'] += [width]
                    information['height'] += [height] 
                    information['file'] += [annotation.split('/')[-1][0:-4]] 
                            
                if 'bndbox' in attribute.tag:
                    for dimension in list(attribute):
                        if 'xmin' in dimension.tag:
                            xmin = int(round(float(dimension.text)))
                            information['xmin'] += [xmin]
                        if 'ymin' in dimension.tag:
                            ymin = int(round(float(dimension.text)))
                            information['ymin'] += [ymin]                                
                        if 'xmax' in dimension.tag:
                            xmax = int(round(float(dimension.text)))
                            information['xmax'] += [xmax]                                
                        if 'ymax' in dimension.tag:
                            ymax = int(round(float(dimension.text)))
                            information['ymax'] += [ymax]

In [10]:
annotations_info_df = pd.DataFrame(information)
annotations_info_df.head(10)

In [11]:
annotations_info_df['annotation_file'] = annotations_info_df['file'] + '.xml'
annotations_info_df['image_file'] = annotations_info_df['file'] + '.png'
 
annotations_info_df.loc[annotations_info_df['label'] == 'mask_weared_incorrect', 'label'] = 'mask_incorrectly_worn'

In [12]:
annotations_info_df

In [13]:
def render_image(image):
    plt.figure(figsize = (12, 8))
    plt.imshow(image)
    plt.show()
    
def convert_to_RGB(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [14]:
annotations_info_df['image_file'].iloc[0]

In [15]:
image_737_path = '../input/face-mask-detection/images/' + annotations_info_df['image_file'].iloc[0]
image_737_path

In [16]:
image_737 = cv2.imread(image_737_path)
image_737

In [17]:
render_image(convert_to_RGB(image_737))

In [18]:
annotation_737_path = '../input/face-mask-detection/annotations/' + annotations_info_df['annotation_file'].iloc[0]
annotation_737_path

In [19]:
image_737.shape

In [20]:
x = annotations_info_df['xmin'].iloc[0]
y = annotations_info_df['ymin'].iloc[0]
width = annotations_info_df['xmax'].iloc[0]
height = annotations_info_df['ymax'].iloc[0]

cropped_737 = image_737[y:height, x:width]
render_image(cropped_737)


In [21]:
render_image(convert_to_RGB(cropped_737))

In [22]:
len(annotations_info_df)


In [23]:
annotations_info_df.head(10)

In [24]:
!ls '../input/face-mask-detection'

In [25]:
directory = 'cropped_images'
parent_directory = '/kaggle/working'
path = os.path.join(parent_directory, directory)
os.mkdir(path)

In [26]:
!ls './'

In [27]:
annotations_info_df['cropped_image_file'] = annotations_info_df['file']
annotations_info_df


In [28]:
for i in range(len(annotations_info_df)):
    image_filepath = '../input/face-mask-detection/images/' + annotations_info_df['image_file'].iloc[i]
    image = cv2.imread(image_filepath)
    annotations_info_df['cropped_image_file'].iloc[i] = annotations_info_df['cropped_image_file'].iloc[i] + '-' + str(i) + '.png'
    cropped_image_filename = annotations_info_df['cropped_image_file'].iloc[i]
    
    xmin = annotations_info_df['xmin'].iloc[i]
    ymin = annotations_info_df['ymin'].iloc[i]
    xmax = annotations_info_df['xmax'].iloc[i]
    ymax = annotations_info_df['ymax'].iloc[i]

    # Crop The Image Based on The Values Above
    cropped_image = image[ymin:ymax, xmin:xmax]
    
    # Save Cropped Image
    cropped_image_directory = os.path.join('./cropped_images', cropped_image_filename) 
    cv2.imwrite(cropped_image_directory, cropped_image)

In [29]:
annotations_info_df

In [30]:
cropped_images_files = !ls './cropped_images'
cropped_images_files[:10]

In [31]:
print('There are {} cropped images in total.'.format(len(cropped_images_files)))

In [32]:
annotations_info_df['cropped_image_file'].iloc[0]

In [33]:
cropped_737_0_path = './cropped_images/' + annotations_info_df['cropped_image_file'].iloc[0]
cropped_737_0_path

In [34]:
cropped_737_0 = cv2.imread(cropped_737_0_path)
cropped_737_0

In [35]:
render_image(convert_to_RGB(cropped_737_0))

In [36]:
cropped_737_0.shape

In [37]:
test_df = annotations_info_df[:800]
train_df = annotations_info_df[800:]
train_df.shape, test_df.shape

In [38]:
train_df.head()

In [39]:
classes = list(train_df['label'].unique())

In [40]:
train_df

In [41]:
train_df[train_df['file'] == 'maksssksksss139']['label'].unique()

In [42]:
image_139_path = '../input/face-mask-detection/images/maksssksksss139.png'
image_139 = cv2.imread(image_139_path)
image_139

In [43]:
image_139_rgb = convert_to_RGB(image_139)
render_image(image_139_rgb)

In [44]:
image_139_df = train_df[train_df['file'] == 'maksssksksss139']
image_139_df

In [45]:
with_mask_list, without_mask_list, incorrectly_worn_list = [], [], []
for i in range(len(image_139_df)):
    bounding_box = [image_139_df['xmin'].iloc[i], image_139_df['ymin'].iloc[i],
                    image_139_df['xmax'].iloc[i], image_139_df['ymax'].iloc[i]]
    if image_139_df['label'].iloc[i] == 'with_mask':
        with_mask_list.append(bounding_box)
    elif image_139_df['label'].iloc[i] == 'without_mask':
        without_mask_list.append(bounding_box)
    else:
        incorrectly_worn_list.append(bounding_box)
        
found_objects_dict = {'With Mask': with_mask_list, 
                      'Without Mask': without_mask_list, 
                      'Incorrectly Worn': incorrectly_worn_list}
found_objects_dict

In [46]:
for key, value in found_objects_dict.items():
    for i in range(len(value)):
        color = (0, 255, 0) # green
        text = 'Mask'
        if key == 'Without Mask':
            color = (255, 0, 0) # red
            text = 'No Mask'
        elif key == 'Incorrectly Worn':
            color = (255, 255, 0) # yellow
            text = 'Incorrect'
        start_point = (value[i][0], value[i][1])
        end_point = (value[i][2], value[i][3])
        cv2.rectangle(image_139_rgb, start_point, end_point, color = color, thickness = 2)
        cv2.putText(image_139_rgb, org = (value[i][0] - 8, value[i][1] - 3), text = text, 
                    fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.5, color = color)

In [47]:
render_image(image_139_rgb)

In [48]:
train_df['label'].value_counts()

In [49]:
sorted_label_df = pd.DataFrame(train_df['label'].value_counts()).reset_index()
sorted_label_df.rename(columns = {'index': 'label', 'label': 'count'}, inplace = True)
sorted_label_df

In [50]:
plt.style.use('seaborn')
plt.figure(figsize = (8, 6))
barplot = sns.barplot(x = 'count', y = 'label', data = sorted_label_df, orient = 'horizontal', 
                      palette = ['green', 'red', 'yellow'])
plt.title('Distribution of Labels', fontsize = 20, fontweight = 'bold')
plt.xlabel('Count', fontsize = 15, fontweight = 'bold')
plt.ylabel('Label', fontsize = 15, fontweight = 'bold')

for p in barplot.patches:
    width = p.get_width()
    percentage = round(width * 100 / sum(sorted_label_df['count']), 2)
    plt.text(x = width + 15, y = p.get_y() + 0.55 * p.get_height(), s = f'{int(width)}\n({percentage} %)')

plt.show()

In [51]:
image_width = []
image_height = []
for i in range(len(train_df)):
    cropped_image_path = './cropped_images/' + train_df['cropped_image_file'].iloc[i]
    cropped_image = cv2.imread(cropped_image_path)
    image_width.append(cropped_image.shape[0])
    image_height.append(cropped_image.shape[1])

In [52]:
image_target_size = (int(np.median(image_width)), int(np.median(image_height)))
image_target_size

In [53]:
from keras_preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(rescale = 1. / 255., validation_split = 0.25)

train_generator = train_image_generator.flow_from_dataframe(
    dataframe = train_df,
    directory = './cropped_images',
    x_col = 'cropped_image_file',
    y_col = 'label',
    subset = 'training',
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = 'categorical',
    target_size = image_target_size
)

valid_generator = train_image_generator.flow_from_dataframe(
    dataframe = train_df,
    directory = './cropped_images',
    x_col = 'cropped_image_file',
    y_col = 'label',
    subset = 'validation',
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = 'categorical',
    target_size = image_target_size
)

In [54]:
test_image_generator = ImageDataGenerator(rescale = 1. / 255.)

test_generator = train_image_generator.flow_from_dataframe(
    dataframe = test_df,
    directory = './cropped_images',
    x_col = 'cropped_image_file',
    y_col = 'label',
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = 'categorical',
    target_size = image_target_size
)

In [55]:
print(train_generator)
print(valid_generator)
print(test_generator)

In [56]:
input_shape = [int(np.median(image_width)), int(np.median(image_height)), 3]

In [57]:
model_1 = keras.models.Sequential([
    keras.layers.Conv2D(filters = 10, kernel_size = 3, activation = 'relu', 
                        input_shape = input_shape),
    keras.layers.Conv2D(filters = 10, kernel_size = 3, activation = 'relu'),
    keras.layers.MaxPool2D(pool_size = 2, padding = 'valid'),
    keras.layers.Conv2D(filters = 10, kernel_size = 3, activation = 'relu'),
    keras.layers.Conv2D(filters = 10, kernel_size = 3, activation = 'relu'),
    keras.layers.MaxPool2D(pool_size = 2, padding = 'valid'),
    keras.layers.Flatten(),
    keras.layers.Dense(units = len(classes), activation = 'softmax')
])

In [58]:
model_1.compile(loss = 'categorical_crossentropy',
                optimizer = keras.optimizers.Adam(),
                metrics = ['accuracy', keras.metrics.Recall()])

history_1 = model_1.fit(train_generator, epochs = 10, steps_per_epoch = len(train_generator), 
                        validation_data = valid_generator, validation_steps = len(valid_generator))